In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import random
from random import choices
from scipy import stats
import pickle
from scipy.stats import chi2_contingency
from sklearn.metrics import matthews_corrcoef
import matplotlib.font_manager as font_manager
from statsmodels.stats.diagnostic import lilliefors

In [ ]:
data = pd.read_excel('drive/MyDrive/Dissertation/data/train_data_fixedltcs.xlsx')

In [ ]:
toremove = ['eid', 'first_admi_date','first_discharge_date', 'second_admi_date', 'delta_length', 'admi_meth']
data.drop(toremove, axis = 1, inplace = True)

In [ ]:
admission = list(data.columns)[0]
target = list(data.columns)[1]
demographics = list(data.columns)[2:12]
hist_drugs =  list(data.columns)[12:508]
pread_drugs = list(data.columns)[508:846]
ltcs = list(data.columns)[846:1047]
diags = list(data.columns)[1047:]

In [ ]:
print(admission)
print(target)
print(demographics[0],demographics[-1])
print(hist_drugs[0],hist_drugs[-1])
print(pread_drugs[0],pread_drugs[-1])
print(ltcs[0],ltcs[-1])
print(diags[0],diags[-1])

first_admi_duration
readmission_state
sex body_fat_percentage
hist_9003020 hist_1906010
pread_1404150 pread_0706000
ltc_PMR ltc_SARS_CoV2
diag_A099 diag_Z466


In [ ]:
binary = ['sex'] + hist_drugs + pread_drugs + ltcs + diags
target = target
continuous = ['age_at_event','first_admi_duration','ltc_count', 'townsend_deprivation_index','alcohol_intake_frequency', 'smoking_status',
              'relative_smoking_pack_years', 'ethnic_background','BMI', 'body_fat_percentage']

In [ ]:
print(len(binary), len(continuous))

# Statistical Analysis

## Correlation study



In [ ]:
def correlation_study(df, continuous, binary, target):
    '''
    Generate correlation matrix considering different correlation coefficients for mixed type data.
    Pearson: numerical-numerical
    Point-biserial: numerical-binary/categorical
    Matthews: binary-binary
    '''
    corr_vals = []
    variables = [target] + binary + continuous
    binary.append(target)

    for var1 in variables:
        listvar1 = []
        for var2 in variables:
            if var1 in continuous and var2 in continuous:
                corr, _ = stats.pearsonr(df[var1], df[var2])
                listvar1.append(corr)
            if var1 in continuous and var2 in binary:
                pbc, pval = stats.pointbiserialr(df[var2], df[var1])
                listvar1.append(pbc)
            if var1 in binary and var2 in continuous:
                pbc, pval = stats.pointbiserialr(df[var2], df[var1])
                listvar1.append(pbc)
            if var1 in binary and var2 in binary:
                listvar1.append(matthews_corrcoef(df[var2], df[var1]))
        corr_vals.append(listvar1)

    corrdf = pd.DataFrame(corr_vals, columns = variables)
    corrdf['Var'] = variables
    corrdf.set_index('Var', inplace = True)
    corrdf.update(corrdf.abs())
    return corrdf, corr_vals

corrdf, corr_vals = correlation_study(data, continuous, binary, target)

## Inference - Feature Selection

In [ ]:
def two_proportion_test(len1, prop1, len2, prop2, variable):
    n1 = len1
    n2 = len2
    n = n1+n2
    p1 = prop1
    p2 = prop2
    x1 = p1*n1
    x2 = p2*n2
    p_pool = (x1+x2)/n
    se = np.sqrt(p_pool*(1-p_pool)*(1/n1+1/n2))
    z_statistic = (p1-p2)/se
    p_value = 2*stats.norm().cdf(-1*np.abs(z_statistic))
#     len1, prop1 = df[df[var]==1].shape[0], df[(df[var]==1) & (df[target]==True)].shape[0]/df[df[var]==1].shape[0]
#     len2, prop2 = df[df[var]==1].shape[0], df[(df[var]==1) & (df[target]==False)].shape[0]/df[df[var]==1].shape[0]
#     pval = two_proportion_test(len1, prop1, len2, prop2, variable = i)
    return p_value

def comparison(df,continuous, binary, target, p_val_thresh):
    selected = []
    pvals = []

    for i in binary:
        if len(list(set(list(df[i].values)))) != 1:
          # # If var is binary perform Chi2 test.
          # contingency = pd.crosstab(df[target], df[i])
          # res = stats.chi2_contingency(contingency)
          # if res.pvalue <= p_val_thresh:
          #     selected.append(i)
          #     pvals.append(res.pvalue)
          len1, prop1 = df[df[i]==1].shape[0], df[(df[i]==1) & (df[target]==True)].shape[0]/df[df[i]==1].shape[0]
          len2, prop2 = df[df[i]==1].shape[0], df[(df[i]==1) & (df[target]==False)].shape[0]/df[df[i]==1].shape[0]
          pval = two_proportion_test(len1, prop1, len2, prop2, variable = i)
          if pval <= p_val_thresh:
            selected.append(i)
            pvals.append(pval)

    for i in continuous:
        # Check for normality of the distributions. Cannot use shapiro for increased N. Cannot use Kolmogorov because
        # we do not know the exact normal distribution it should come from. We use Lillieforths test (estimated params
        # version of Kolmogorov).
        _, pval1 = lilliefors(df[i][df[target]==True], dist='norm', pvalmethod='table')
        _, pval2 = lilliefors(df[i][df[target]==False], dist='norm', pvalmethod='table')

        if (pval1 > 0.05) and (pval2 > 0.05):
            # if variable is normal, perform two-sample 2 sided t-test.
            stat, p = stats.bartlett(df[i][df[target]==True], df[i][df[target]==False])

            if p <= 0.05:
                # Welch´s t-test does not assume equal variances.
                _, pv = stats.ttest_ind(df[i][df[target]==True], df[i][df[target]==False], equal_var = False, alternative = 'two-sided')
                if pv <= p_val_thresh:
                    selected.append(i)
                    pvals.append(pval)
            else:
                _, pv = stats.ttest_ind(df[i][df[target]==True], df[i][df[target]==False], equal_var = True, alternative = 'two-sided')
                if pv <= p_val_thresh:
                    selected.append(i)
                    pvals.append(pval)
        else:
            # If variable is not normal, perform MW-U test, non parametric analogous to t-test.
            tval, pval = stats.mannwhitneyu(df[i][df[target]==True], df[i][df[target]==False], alternative = 'two-sided')
            if pval <= p_val_thresh:
                selected.append(i)
                pvals.append(pval)

  return selected, pvals

In [ ]:
selected, pvals = comparison(data,continuous, binary, 'readmission_state', 0.05)

selection2prop = pd.DataFrame()
selection2prop['selected'] = selected
selection2prop['pvals'] = pvals
selection2prop.to_excel('drive/MyDrive/Dissertation/data/selection2prop005_imbalanced data.xlsx', index = False)